아주 쉽고 재미있게, 직관적으로 딥러닝 개념을 설명한 하용호님의 자료를 공유합니다. (주의)'한 번 클릭하면 10분~15분동안 다른 일을 할 수가 없습니다.'

![발표자료 링크](https://www.slideshare.net/yongho/ss-79607172)

![img](http://tykimos.github.com/Keras/warehouse/2017-9-16-YonghoHa_Slide_01.png)

이 발표자료에 사용된 케라스 코드를 구현해보고, 딥브릭으로 쌓아보겠습니다.

---
### 레이어 준비

영상입력 수치예측 모델에서 사용할 레이어는 

수치예측 모델에 사용할 레이어는 `Dense`와 `Activation`입니다. `Activation`에는 은닉층(hidden layer)에 사용할 `relu`를 준비했습니다. 데이터셋은 일차원 벡터만 다루도록 하겠습니다.

|종류|구분|상세구분|브릭|
|:-:|:-:|:-:|:-:|
|데이터셋|Vector|-|![img](http://tykimos.github.com/Keras/warehouse/DeepBrick/Model_Recipe_Part_Dataset_Vector_s.png)|
|레이어|Dense||![img](http://tykimos.github.com/Keras/warehouse/DeepBrick/Model_Recipe_Part_Dense_s.png)|
|레이어|Activation|relu|![img](http://tykimos.github.com/Keras/warehouse/DeepBrick/Model_Recipe_Part_Activation_Relu_s.png)|

---
### VGG16 모델

i발표자료에 포함된 VGG16 모델은 다음과 같습니다. 케라스 버전 및 딥브릭 적용을 위해 소스를 약간 수정했습니다.

#### 컨볼루션 레이어 블록 정의

In [ ]:
def ConvBlock(layers, model, filters):
    for i in range(layers):
        model.add(Conv2D(filters, (3, 3), activation='relu'))
    model.add(MaxPooling2D(2, 2), strides=(2,2))

In [ ]:
ConvBlock(2, model, 64)

In [ ]:
ConvBlock(3, model, 512)

#### 전결합층 블록 정의

In [ ]:
def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

#### VGG16 모델 정의

In [ ]:
def VGG_16():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,244,244)))
    
    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)
    
    model.add(Flatten())
    FCBlock(model)
    FCBlock(model)
    model.add(Dense(1000, activation='softmax'))
    
    return model

---
### 모델 준비

수치예측을 하기 위해 `다층퍼셉트론 신경망 모델`, `컨볼루션 신경망 모델`을 준비했습니다.

#### 다층퍼셉트론 신경망 모델

    model = Sequential()
    model.add(Dense(256, activation='relu', input_dim = width*height))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1))

#### 컨볼루션 신경망 모델

    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, 1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(1))

---
### 전체 소스

앞서 살펴본 `다층퍼셉트론 신경망 모델`, `컨볼루션 신경망 모델`의 전체 소스는 다음과 같습니다. 

In [ ]:
# 0. 사용할 패키지 불러오기
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D

# 1. 데이터셋 생성하기


# 2. 모델 구성하기

vgg_mean = np.array([123.68, 116.779, 103.939], dtype=np.float32).reshape((3,1,1))

def vgg_preporcess(x):
    x = x - vgg_mean
    return x

# 컨볼루션 레이어 블록 정의
def ConvBlock(layers, model, filters):
    for i in range(layers):
        model.add(Conv2D(filters, (3, 3), padding='same', activation='relu'))
    model.add(MaxPooling2D(2, 2), strides=(2,2))
    
# 전결합층 블록 정의
def FCBlock(model):
    model.add(Dense(4096, activation='relu'))
    model.add(Dropout(0.5))

# VGG16 모델 정의
def VGG_16():
    model = Sequential()
    model.add(Lambda(vgg_preprocess, input_shape=(3,244,244)))
    
    ConvBlock(2, model, 64)
    ConvBlock(2, model, 128)
    ConvBlock(3, model, 256)
    ConvBlock(3, model, 512)
    ConvBlock(3, model, 512)
    
    model.add(Flatten())
    
    FCBlock(model)
    FCBlock(model)
    
    model.add(Dense(1000, activation='softmax'))
    
    return model

model = VGG_16()

# 3. 모델 학습과정 설정하기
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# 4. 모델 학습시키기
hist = model.fit(x_train, y_train, epochs=5, batch_size=64, validation_data(x_val, y_val))


# 5. 학습과정 살펴보기

# 6. 모델 평가하기

# 7. 모델 사용하기

---

### 요약

N/A

---

### 같이 보기

* [강좌 목차](https://tykimos.github.io/Keras/lecture/)